## Currency Conversion

In [85]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_core.tools import InjectedToolArg
from typing import Annotated
import requests
import os 
from dotenv import load_dotenv

EXCHANGERATE_API_KEY = os.getenv("EXCHANGERATE_API_KEY")

load_dotenv()

True

In [86]:
# tool create

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/{EXCHANGERATE_API_KEY}/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate


In [87]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

In [88]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'PKR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1753488001,
 'time_last_update_utc': 'Sat, 26 Jul 2025 00:00:01 +0000',
 'time_next_update_unix': 1753574401,
 'time_next_update_utc': 'Sun, 27 Jul 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'PKR',
 'conversion_rate': 285.1463}

In [89]:
convert.invoke({'base_currency_value':10, 'conversion_rate':85.16})

851.5999999999999

In [90]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

In [91]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [92]:
messages = [HumanMessage('What is the conversion factor between PKR and USD, and based on that can you convert 10 pkr to usd')]

In [93]:
messages

[HumanMessage(content='What is the conversion factor between PKR and USD, and based on that can you convert 10 pkr to usd', additional_kwargs={}, response_metadata={})]

In [94]:
ai_message = llm_with_tools.invoke(messages)

In [95]:
messages.append(ai_message)

In [96]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'target_currency': 'USD', 'base_currency': 'PKR'},
  'id': '92eeaaaf-3b5c-45c7-addf-394077e681f3',
  'type': 'tool_call'}]

In [97]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)
        

In [98]:
messages

[HumanMessage(content='What is the conversion factor between PKR and USD, and based on that can you convert 10 pkr to usd', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_conversion_factor', 'arguments': '{"target_currency": "USD", "base_currency": "PKR"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--2876df4a-812d-4aa6-adb7-0ca40e7a8e73-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'target_currency': 'USD', 'base_currency': 'PKR'}, 'id': '92eeaaaf-3b5c-45c7-addf-394077e681f3', 'type': 'tool_call'}], usage_metadata={'input_tokens': 93, 'output_tokens': 14, 'total_tokens': 107, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='{"result": "success", "documentation": "https://www.exchangerate-api.com/docs", "terms_of_use": "https://www.exchangerate-api.com/terms", "time_last_update_unix": 17534880

In [99]:
llm_with_tools.invoke(messages).content

'The conversion factor between PKR and USD is 0.003507. I cannot convert 10 PKR to USD with the available tools, as the convert function requires a currency conversion rate.'